## This notebook was for the first cut solution of a CNN + for evaluating the different dataset we generated from our data in accordance with the CNN predictions

In [1]:
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding,GlobalMaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
import numpy as np
import pandas as pd
from gensim.models.word2vec import Word2Vec
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical, plot_model
from keras.layers import Dense, Dropout, Conv1D, MaxPool1D, GlobalMaxPool1D, Embedding, Activation
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from sklearn import preprocessing
import pickle
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras_visualizer import visualizer 
import matplotlib.pyplot as plt
from sklearn.metrics import make_scorer, accuracy_score, f1_score
import visualkeras



# Preprocessing and model is used from 
# 

2023-03-07 10:36:48.909334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 10:36:49.070191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 10:36:49.070219: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-07 10:36:49.969201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

## Adapt the preprocessing pipeline to the songs so thath the input is cleaned

In [2]:
import spacy

def tokenization(text):
    """Use this function to tokenize text.

    :param text: Text as list
    :type text: list[spacy.tokens.token.Token]
    :return: Tokenized text as list
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list = []
    for doc in text: 
        # iterate over tokens in docs
        for token in doc:
            token_list.append(token)

    return token_list


def stop_word_removal(text): 
    """Use this function to remove stop words. 

    :param text: Tokens to remove stop words from 
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without stop words
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_without_stop = []
    # Don't add token to list if stop word
    for token in text:
        if token.is_stop == False: 
            token_list_without_stop.append(token)

    return token_list_without_stop


def punctutation_removal(text): 
    """Use this function to remove punctuation.

    :param text: Tokens to remove punctuation from
    :type text: list[spacy.tokens.token.Token]
    :return: Tokens without punctuation
    :rtype: list[spacy.tokens.token.Token]
    """

    token_list_no_stop_no_punct = []
    # Don't add token to list if punctuation
    for token in text:
        if token.is_punct == False:
            token_list_no_stop_no_punct.append(token)

    return token_list_no_stop_no_punct


def lemmatization(text): 
    """Use this function to lemmatize a given text.

    :param text: Tokens to lemmatize
    :type text: list[spacy.tokens.token.Token]
    :return: lemmatized tokens
    :rtype: list[str]
    """

    token_list_no_stop_no_punct_lemmatized = []
    for token in text: 
        if "\n" not in token.lemma_:
            token_list_no_stop_no_punct_lemmatized.append(token.lemma_)
    return token_list_no_stop_no_punct_lemmatized


def processing_pipeline(song_data):
    """Use this function to execute the entire processing pipeline on given song data.
    Preprocessing steps:
    - Tokenization
    - Stop word removal
    - Punctuation removal
    - Lemmatization
    - ...

    :param song_data: song data saved in a json file containing song name, artist name and lyrics
    :type song_data: dict
    :return: preprocessed song data
    :rtype: dict
    """

    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    
    for row in range(len(song_data)):
        text_nlp_pipe = list(nlp.pipe([song_data.iloc[row]["Lyric"]]))
    
        # Tokenization
        song_data.at[row,"Lyric"] = tokenization(text_nlp_pipe)
        # Stop word removal
        song_data.at[row,"Lyric"] = stop_word_removal(song_data.iloc[row]["Lyric"])
        # Punctuation removal
        song_data.at[row,"Lyric"] = punctutation_removal(song_data.iloc[row]["Lyric"])
        # Lemmatization
        song_data.at[row,"Lyric"] = lemmatization(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = " ".join(song_data.iloc[row]["Lyric"])
        song_data.at[row,"Lyric"] = song_data.iloc[row]["Lyric"].lower()
        if row%500 == 0 and row >= 500:
            print(f"processsed: {row} rows out of {len(song_data)}")
    return song_data


2023-03-07 10:36:52.133273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-07 10:36:52.133589: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 10:36:52.133668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-07 10:36:52.133736: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-07 10:36:52.133802: W tensorf

In [3]:
# helper function to use the word2vec as an layer in keras 
# taken from the gensim wikipage: https://github.com/RaRe-Technologies/gensim/wiki/Using-Gensim-Embeddings-with-Keras-and-Tensorflow

from tensorflow.keras.layers import Embedding

def gensim_to_keras_embedding(model, train_embeddings=False):
    """Get a Keras 'Embedding' layer with weights set from Word2Vec model's learned word embeddings.

    Parameters
    ----------
    train_embeddings : bool
        If False, the returned weights are frozen and stopped from being updated.
        If True, the weights can / will be further updated in Keras.

    Returns
    -------
    `keras.layers.Embedding`
        Embedding layer, to be used as input to deeper network layers.

    """
    keyed_vectors = model.wv  # structure holding the result of training
    weights = keyed_vectors.vectors  # vectors themselves, a 2D numpy array    
    index_to_key = keyed_vectors.index_to_key  # which row in `weights` corresponds to which word?

    layer = Embedding(
        input_dim=weights.shape[0],
        output_dim=weights.shape[1],
        weights=[weights],
        trainable=train_embeddings,
    )
    return layer

In [4]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def create_model(density, word2vecmodel,mood_count,filters,multiplicator, multiplicator2, kernel, outputfunction):
    keras_model = Sequential()
    keras_model.add(gensim_to_keras_embedding(word2vecmodel, True))
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(MaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(Conv1D(filters*multiplicator2, kernel, activation='relu', padding='same', strides=1))
    keras_model.add(GlobalMaxPool1D())
    keras_model.add(Dropout(0.2))
    keras_model.add(Dense(density))
    keras_model.add(Activation('relu'))
    keras_model.add(Dropout(0.2))
    # Number of moods to be classified to
    keras_model.add(Dense(mood_count))
    keras_model.add(Activation(outputfunction))
    
    keras_model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
    
    return keras_model


In [5]:
def print_results(grid_result):
    
    # get the results of GridSearchCV
    results = grid_result.cv_results_

    # create a list of the parameter settings for the x-axis
    x_axis = range(len(results['params']))

    # create a list of the mean test scores for the y-axis
    y_axis = results['mean_test_score']

    # create a bar chart of the mean test scores
    plt.bar(x_axis, y_axis, align='center')

    # set the x-axis tick labels to the parameter settings
    plt.xticks(x_axis, results['params'], rotation=90)

    # add a title and axis labels
    plt.title('GridSearchCV Results')
    plt.xlabel('Parameter Settings')
    plt.ylabel('Accuracy')

    # display the graph
    plt.show()

In [6]:

def prepare_dataset(dataset_path, mood_count):
    print(f"Testing now dataset {dataset_path} with {mood_count} moods")
    df = pd.read_csv(dataset_path)
    print((df["Mood"].unique()))
    df = processing_pipeline(df)
    list(df["Lyric"].head(2))
    # merge lyrics together
    lyrics = []
    for i in df['Lyric']:
        lyrics.append(i.split())
    print(lyrics[:2])

    # train the word2vec model
    # vector size according to https://moj-analytical-services.github.io/NLP-guidance/NNmodels.html#:~:text=The%20standard%20Word2Vec%20pre%2Dtrained,fewer%20dimensions%20to%20represent%20them.
    # mincount = 2 to prevent misspellings
    word2vec_model = Word2Vec(lyrics, vector_size=150, window=5, min_count=2, workers=16)

    # use the keras tokenizer and apply it to the lyrics
    # number in first row is vocab size from word2vec model
    token = Tokenizer(len(word2vec_model.wv))
    token.fit_on_texts(df['Lyric'])
    text = token.texts_to_sequences(df['Lyric'])
    text = pad_sequences(text, 180)
    print(text[3:5])

    # encode the labels
    le = preprocessing.LabelEncoder()
    y = le.fit_transform(df['Mood'])
    y = to_categorical(y)
    # save the label encoder
    np.save('data/label_encoder.npy', le.classes_)

    x_train, x_test, y_train, y_test = train_test_split(np.array(text), y, test_size=0.01, stratify=y)

    
    return  word2vec_model,x_train, x_test, y_train, y_test
    
def test_dataset(word2vec_model,x_train, x_test, y_train, y_test, mood_count):
    seed = 7
    np.random.seed(seed)
    batch_size = 32
    epochs = 3

    model_CV = KerasClassifier(build_fn=create_model, epochs=epochs, 
                                   batch_size=batch_size, verbose=1)
    # grid search parameters
    filters = [50, 100]
    multiplicator = [2,3]
    multiplicator2 = [4,5]
    kernel = [5]
    outputfunction = ["relu","softmax"]
    word2vecmodel = [word2vec_model]
    density = [200, 100]
                      
    scoring = {'accuracy': make_scorer(accuracy_score), 'f1': make_scorer(f1_score, average="weighted")}

    param_grid = {
        "density": density,
        "word2vecmodel": word2vecmodel,
        "mood_count": [mood_count],
        "filters": filters,
        "multiplicator": multiplicator,
        "multiplicator2": multiplicator2,
        "kernel": kernel,
        "outputfunction": outputfunction
        }
    grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, n_jobs=-1,cv=2, error_score="raise")
    grid_result = grid.fit(x_train, y_train)
    
    # print results
    print(f'Best Accuracy is: {grid_result.best_score_} using {grid_result.best_params_}')
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print(f' mean={mean:.4}, std={stdev:.4} using {param}')
    print("##"*50)
    
    print_results(grid_result)

In [7]:
word2vec_model_7,x_train, x_test, y_train, y_test = prepare_dataset("./data/song-data-labels-cleaned-seven-moods.csv",7)


Testing now dataset ./data/song-data-labels-cleaned-seven-moods.csv with 7 moods
['romantic' 'happy' 'sad' 'enthusiastic' 'calm' 'depressed' 'anger']
processsed: 500 rows out of 14308
processsed: 1000 rows out of 14308
processsed: 1500 rows out of 14308
processsed: 2000 rows out of 14308
processsed: 2500 rows out of 14308
processsed: 3000 rows out of 14308
processsed: 3500 rows out of 14308
processsed: 4000 rows out of 14308
processsed: 4500 rows out of 14308
processsed: 5000 rows out of 14308
processsed: 5500 rows out of 14308
processsed: 6000 rows out of 14308
processsed: 6500 rows out of 14308
processsed: 7000 rows out of 14308
processsed: 7500 rows out of 14308
processsed: 8000 rows out of 14308
processsed: 8500 rows out of 14308
processsed: 9000 rows out of 14308
processsed: 9500 rows out of 14308
processsed: 10000 rows out of 14308
processsed: 10500 rows out of 14308
processsed: 11000 rows out of 14308
processsed: 11500 rows out of 14308
processsed: 12000 rows out of 14308
proces

In [8]:
test_dataset(word2vec_model_7,x_train, x_test, y_train, y_test, 7)

/tmp/ipykernel_7653/2941263412.py:44: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model_CV = KerasClassifier(build_fn=create_model, epochs=epochs,
2023-03-07 10:43:08.096782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 10:43:08.123395: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 10:43:0

2023-03-07 10:43:09.832460: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 10:43:09.833431: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 10:43:09.833570: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-03-07 10:43:09.879221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory


2023-03-07 10:43:11.789830: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-07 10:43:11.790647: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 10:43:11.790857: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-07 10:43:11.791149: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-07 10:43:11.791291: W tensorf

Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
Epoch 1/3
222/222 [==============================] - 126s 550ms/step - loss: 1.1264 - accuracy: 0.1466
Epoch 2/3
222/222 [==============================] - 126s 548ms/step - loss: 1.0365 - accuracy: 0.1439
Epoch 2/3
222/222 [==============================] - 129s 558ms/step - loss: 0.4226 - accuracy: 0.1418
Epoch 2/3
222/222 [==============================] - 129s 547ms/step - loss: 0.4223 - accuracy: 0.1443
Epoch 2/3
222/222 [==============================] - 140s 615ms/step - loss: 0.4213 - accuracy: 0.1466
Epoch 2/3
222/222 [==============================] - 147s 631ms/step - loss: 0.4822 - accuracy: 0.1409
Epoch 2/3
222/222 [==============================] - 125s 565ms/step - loss: 0.4106 - accuracy: 0.1742
Epoch 3/3
Epoch 3/3
222/222 [==============================] - 129s 580ms/step - loss: 0.4138 - accuracy: 0.1478
Epoch 3/3
222/222 [==============================] - 129s 583ms/step - loss: 0.4140 - accuracy: 

222/222 [==============================] - 315s 1s/step - loss: 5.2728 - accuracy: 0.1419
Epoch 2/3
222/222 [==============================] - 300s 1s/step - loss: 0.4235 - accuracy: 0.1354
Epoch 2/3
222/222 [==============================] - 297s 1s/step - loss: 3.0727 - accuracy: 0.1425
Epoch 3/3
222/222 [==============================] - 409s 2s/step - loss: 2.6074 - accuracy: 0.1461
Epoch 2/3
222/222 [==============================] - 294s 1s/step - loss: 3.0785 - accuracy: 0.1494
Epoch 3/3
222/222 [==============================] - 427s 2s/step - loss: 3.7537 - accuracy: 0.1430
Epoch 2/3
222/222 [==============================] - 430s 2s/step - loss: 0.4190 - accuracy: 0.1426
Epoch 2/3
222/222 [==============================] - 442s 2s/step - loss: 0.4231 - accuracy: 0.1497
Epoch 2/3
222/222 [==============================] - 301s 1s/step - loss: 0.4123 - accuracy: 0.1665
Epoch 3/3
222/222 [==============================] - 304s 1s/step - loss: 0.4131 - accuracy: 0.1488
Epoch 3/3


...

  7/222 [..............................] - ETA: 27s - loss: 0.4097 - accuracy: 0.1652

2023-03-07 12:04:05.286782: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-07 12:04:05.288552: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-07 12:04:05.288920: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


 58/222 [======>.......................] - ETA: 1:31 - loss: 0.4059 - accuracy: 0.1983

2023-03-07 12:04:17.601067: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-07 12:04:17.603809: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-07 12:04:17.603983: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-03-07 12:04:17.604094: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-03-07 12:04:17.604197: W tensorf

222/222 [==============================] - 301s 1s/step - loss: 1.6896 - accuracy: 0.1361
Epoch 2/3
222/222 [==============================] - 299s 1s/step - loss: 0.7422 - accuracy: 0.1445
Epoch 2/3
222/222 [==============================] - 292s 1s/step - loss: 0.4273 - accuracy: 0.1452
Epoch 2/3
222/222 [==============================] - 324s 1s/step - loss: 0.4288 - accuracy: 0.1457
Epoch 2/3
222/222 [==============================] - 300s 1s/step - loss: 0.5147 - accuracy: 0.1437
Epoch 3/3
222/222 [==============================] - 430s 2s/step - loss: 1.5341 - accuracy: 0.1402
Epoch 2/3
222/222 [==============================] - 437s 2s/step - loss: 1.0111 - accuracy: 0.1457
Epoch 2/3
222/222 [==============================] - 294s 1s/step - loss: 0.4130 - accuracy: 0.1412
Epoch 3/3
222/222 [==============================] - 292s 1s/step - loss: 0.4165 - accuracy: 0.1507
Epoch 3/3
222/222 [==============================] - 440s 2s/step - loss: 0.4252 - accuracy: 0.1477
Epoch 2/3


In [ ]:
word2vec_model_17,x_train, x_test, y_train, y_test = prepare_dataset("./data/song-data-labels-cleaned.csv",17)


In [ ]:
test_dataset(word2vec_model_17,x_train, x_test, y_train, y_test, 17)

In [ ]:
word2vec_model_4,x_train, x_test, y_train, y_test = prepare_dataset("./data/song-data-labels-cleaned-quadrant.csv",4)


In [ ]:
test_dataset(word2vec_model_4,x_train, x_test, y_train, y_test, 4)

In [ ]:
keras_model = create_model(200, word2vec_model_17,7, 50, 2, 4, 5, "softmax")
plot_model(
    keras_model,
    to_file="model.png",
    show_shapes=False,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
)